In [2]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn import preprocessing
import matplotlib.pyplot as plt
min_max_scaler=preprocessing.MinMaxScaler()
import warnings
warnings.filterwarnings('ignore')
import glob

In [3]:
maxNoOfCircle = 30

In [4]:
nodeLookUp={}
nodeIdx=-1
def getNodeId(rawId):
    global nodeIdx
    if rawId in nodeLookUp:
        nodeId = nodeLookUp[rawId]
    else:
        nodeIdx=1+nodeIdx
        nodeId = nodeIdx
        nodeLookUp[rawId] = nodeId
    return nodeId

In [5]:
noOfNodes=100
noOfTestSamples=int(0.2*noOfNodes)
testSetExpected=np.zeros((noOfNodes,noOfNodes))
testSetIdxs=np.random.randint(0,noOfNodes-1,noOfTestSamples)

def initForCircle():
    global nodeLookUp
    global nodeIdx
    nodeLookUp={}
    nodeIdx=-1
    

In [6]:
def loadCircle(circleFile):
    global noOfTestSamples
    global testSetExpected
    global testSetIdxs
    global nodeLookUp
    global noOfNodes
    circles = open(circleFile, 'r')
    for circle in circles:
        nodesInCircle = circle.split()[1:]
        for node in nodesInCircle:
            getNodeId(node)
    noOfNodes=len(nodeLookUp)
    print 'noOfNodes:',noOfNodes
    noOfTestSamples=int(noOfNodes/5)
    testSetExpected=np.zeros((noOfNodes,noOfNodes))
    testSetIdxs=np.random.randint(0,noOfNodes-1,noOfTestSamples)


In [7]:
def bigClam(utilMat,noOfCircles):
    global noOfNodes
    global nodeLookUp
    error1={}
    error2={}
    P_C={}
    ssd={}
    testSSE={}
    earlyStop=-1
    for k in range(1,maxNoOfCircle):
        P_C[k]=0
        #print 'to fit util mat with k',utilMat.shape,k
        model = NMF(n_components=k,nls_max_iter=10000,solver='cd',l1_ratio=1,alpha=0.7,init='nndsvd',tol=0.000001)
        F = model.fit_transform(utilMat)
        error1[k]=model.reconstruction_err_
        error2[k]=0
        nodeCircleProbs = np.zeros((noOfNodes,k))
        for i in range(k):
            circleModel = NMF(n_components=1,solver='cd',l1_ratio=0,alpha=1,tol=0.000001,init='nndsvd',nls_max_iter=10000,max_iter=10000)
            F_=F[:,i].reshape(len(F[:,i]),1)
            F_ab_=F_*np.transpose(F_)
            min_max_scaler.fit([0, 100])
            neg_F_ab_=-1*min_max_scaler.transform(F_ab_)*100
            exp_F_ab_=np.exp(neg_F_ab_)
            nodeCircleProbPrime = min_max_scaler.fit_transform(circleModel.fit_transform(exp_F_ab_))
            nodeCircleProb=1-nodeCircleProbPrime
            nodeCircleProbs[:,i]=nodeCircleProb.reshape(len(nodeLookUp))
            P_c=reduce(lambda y,x: np.log(np.maximum(x,0.00000000000000000000000001))+np.log(np.maximum(1-x,0.0000000000000000000001)),nodeCircleProb)
            P_C[k]=P_C[k]+P_c
            error2[k]=error2[k]+circleModel.reconstruction_err_
        reconstructedUtilMat=np.zeros((noOfNodes,noOfNodes))
        testSetActual=np.zeros((noOfNodes,noOfNodes))
        hardNodeInCircle = nodeCircleProbs > 0.5
        hardNodeInCircle = hardNodeInCircle.astype(int)
        for n1 in range(noOfNodes):
            for n2 in range(noOfNodes):
                if (n1 in testSetIdxs) and (n2 in testSetIdxs):
                    testSetActual[n1,n2] = 1 - np.prod(1-np.multiply(hardNodeInCircle[n1],hardNodeInCircle[n2]))
                    #print testSetActual[n1,n2]
        testSSE[k]=np.sum((testSetExpected-testSetActual)**2)
        for n1 in range(noOfNodes):
            for n2 in range(noOfNodes):
                reconstructedUtilMat[n1,n2]=1 - np.prod(1-np.multiply(hardNodeInCircle[n1],hardNodeInCircle[n2]))
        ssd[k]=np.sum(((utilMat/100)-reconstructedUtilMat)**2)
        #print k""",P_C[k],error1[k],error2[k]""",ssd[k],"""np.sum(nodeCircleProbs),"""testSSE[k],np.sum(testSetActual)
        #print k,ssd[k],testSSE[k],np.sum(testSetActual)
        if earlyStop==-1:
            earlyStop = ssd[k]
        elif ssd[k]>earlyStop:
            break
    print np.argmin(ssd),np.argmin(testSSE)
    return np.argmin(ssd),np.argmin(testSSE)
    

In [8]:
def loadUtilMatrix(circleFile):
    global nodeLookUp
    print 'circleFile:',circleFile
    utilMat = np.zeros((len(nodeLookUp),len(nodeLookUp)))
    circles = open(circleFile, 'r')
    np.set_printoptions(threshold=1000)
    noOfCircles=0
    for circle in circles:
        noOfCircles=noOfCircles+1
        nodesInCircle = circle.split()
        nodesInCircle = nodesInCircle[1:]
        print len(nodesInCircle)
        for node1 in nodesInCircle:
            for node2 in nodesInCircle:
                node1Idx=getNodeId(node1)
                node2Idx=getNodeId(node2)
                lnPab_=100
                if (node1Idx in testSetIdxs) and (node2Idx in testSetIdxs):
                    lnPab_=0.69314718056 #P_ab = 0.5
                    testSetExpected[node1Idx,node2Idx]=1
                utilMat[node1Idx,node2Idx]=lnPab_
                utilMat[node1Idx,node2Idx]=utilMat[node1Idx,node2Idx]
    print 'circles.size',noOfCircles
    return utilMat,noOfCircles


In [10]:
for circleFile in glob.glob('../data/circles/*.circles'):
    initForCircle()
    loadCircle(circleFile)
    utilMat,noOfCircles = loadUtilMatrix(circleFile)
    k_ssd,k_testSSE=bigClam(utilMat,noOfCircles)
    print 'circle - actual,predicted',noOfCircles,k_ssd,k_testSSE

In [11]:
%matplotlib inline

def plotHist(dictValues):
    plt.bar(range(len(dictValues)), dictValues.values())
    plt.xticks(range(len(dictValues)), dictValues.keys())
    plt.show()

#plotHist(P_C)
plotHist(ssd)
plotHist(testSSE)



NameError: name 'ssd' is not defined